In [1]:
import cv2
import mediapipe as mp
import math

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distancia (p1, p2):
    return math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2 + (p2.z - p1.z)**2)

In [4]:
# Funcion para determinar si un dedo esta levantado
def dedo_levantado(landmarks, dedo_tip, dedo_dip, dedo_pip, dedo_mcp, umbral=0.1):
    # Comparar la posicion "y" del punto a la punta con los puntos anteriores
    return (landmarks[dedo_tip].y < landmarks[dedo_dip].y and landmarks[dedo_dip].y < landmarks0[dedo_pip].y and landmarks[dedo_pip].y < landmarks[dedo_mcp].y)

In [5]:
# Configurar la captura de la webcam
cap = cv2.VideoCapture(0) # 0 para la camara predeterminada

In [ ]:
with mp_hands.Hands(
    static_image_mode = True,
    max_num_hands = 2,
    min_detection_confidence = 0.5
    min_tracking_confidence = 0.5) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("No se pudo capturar la imagen")
            continue
    
    # image = cv2.imread(ruta)
    # height, width, _ = image.shape
    image = cv2.flip(image,1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    print('Handedness', results.multi_handedness)

    if results.multi_hand_landmarks:
        for hands_landmarks in results.multi_hand_landmarks:
            # Dibujar las marcas y conexiones
            mp_drawing.draw_landmarks(image, hands_landmarks, mp_hands.HAND_CONNECTIONS)
            # Obtener las marcas como una lista
            landmarks = hands_landmarks.landmark

            #definir los indices de los puntos de los dedos (segun mediapipe)
            PULGAR_TIP = 4
            INDICE_TIP = 8
            MEDIO_TIP = 12
            ANULAR_TIP =16
            MENIQUE_TIP = 20

            # PUNTOS PARA CADA DEDO (tip, dip, pip, mcp)
            dedos = {
                'Pulgar':(PULGAR_TIP, 2,1,0),
                'Indice':(INDICE_TIP, 6,5,0),
                'Medio': (MEDIO_TIP, 10, 9, 0),
                'Anular': (ANULAR_TIP, 14, 13,0),
                'Menique': (MENIQUE_TIP, 18,17,0)
            }

            # Verificar que dedos estan levantados
            dedos_levantados = []
            for nombre,(tip,dip, pip, mcp) in dedos.items():
                if dedo_levantado(landmarks, tip, dip, pip, mcp):
                    dedos_levantados.append(nombre)

            # print('Dedos levantados:', dedos_levantados)

            # Mostrar los dedos levantados en la imagen
            cv2.putText(image, f'Dedos: {", ".join(dedos_levantados)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    #MOSTRAR LA IMAGEN RESULTANTE
    # image = cv2.resize(image, (1000, 800))  # Ajusta a un tamaño más pequeño
    cv2.imshow('Imagen', image)
    # cv2.waitKey(0)
    #cv2.destroyAllWindows()
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()